Penjelasan hill_encrypt

- Mengubah urutan angka plaintext menjadi matriks dengan jumlah kolom sesuai dengan panjang key matrix

`plaintext_matrix = np.array(plaintext_numbers).reshape(-1, len(key_matrix))`

Contoh:

"HELLO WORLD" (tanpa spasi) dan matriks kunci 3x3 berikut:

key_matrix = [[1, 2, 3],
              [4, 5, 6],
              [7, 8, 9]]

Urutan angka untuk "HELLO WORLD" adalah [8, 5, 12, 12, 15, 23, 15, 18, 12, 4].

[[ 8  5 12]
 [12 15 23]
 [15 18 12]
 [ 4  0  0]]

Dalam contoh ini, jumlah karakter dalam plaintext tidak habis dibagi dengan panjang matriks kunci, sehingga dua karakter terakhir (D dan spasi) ditempatkan pada baris terakhir, dan nilai 0 ditambahkan untuk melengkapi matriks.

- Melakukan perkalian matriks plaintext dengan key matrix, kemudian diambil modulus 26

`encrypted_matrix = np.dot(plaintext_matrix, key_matrix) % 26`



In [3]:
!pip install numpy sympy

  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
  Using cached sympy-1.12-py3-none-any.whl.metadata (12 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl (15.5 MB)
Using cached sympy-1.12-py3-none-any.whl (5.7 MB)
Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)


In [4]:
!pip list

Package           Version
----------------- -----------
asttokens         2.4.1
colorama          0.4.6
comm              0.2.2
debugpy           1.8.1
decorator         5.1.1
executing         2.0.1
ipykernel         6.29.4
ipython           8.24.0
jedi              0.19.1
jupyter_client    8.6.1
jupyter_core      5.7.2
matplotlib-inline 0.1.7
mpmath            1.3.0
nest-asyncio      1.6.0
numpy             1.26.4
packaging         24.0
parso             0.8.4
pip               24.0
platformdirs      4.2.1
prompt-toolkit    3.0.43
psutil            5.9.8
pure-eval         0.2.2
Pygments          2.18.0
python-dateutil   2.9.0.post0
pywin32           306
pyzmq             26.0.3
six               1.16.0
stack-data        0.6.3
sympy             1.12
tornado           6.4
traitlets         5.14.3
wcwidth           0.2.13


In [7]:
import numpy as np
from sympy import Matrix


# Fungsi untuk menghitung GCD (Greatest Common Divisor)
def gcd(a, b):
    while b:
        a, b = b, a % b
    return a


# Fungsi untuk mengubah teks menjadi angka berdasarkan A=1, B=2, ..., Z=26
# Setiap karakter dalam teks tersebut diubah menjadi angka menggunakan fungsi
# ord(char) - 64. Fungsi ord(char) mengembalikan nilai Unicode dari karakter,
# dan karena ‘A’ dalam Unicode adalah 65, kita kurangi dengan 64 untuk
# mendapatkan 1 untuk ‘A’, 2 untuk ‘B’, dan seterusnya hingga 26 untuk ‘Z’.
def text_to_numbers(text):
    return [(ord(char) - 65) % 26 + 1 for char in text.upper() if char != " "]


# Fungsi untuk mengubah angka menjadi teks berdasarkan 1=A, 2=B, ..., 26=Z
def numbers_to_text(numbers, spaces):
    text = "".join(chr((num - 1) % 26 + 65) for num in numbers)
    for index in spaces:
        text = text[:index] + " " + text[index:]
    return text


# Fungsi untuk enkripsi dengan Hill Cipher
def hill_encrypt(plaintext, key_matrix):
    spaces = [i for i, char in enumerate(plaintext) if char == " "]
    plaintext = plaintext.replace(" ", "")

    plaintext_numbers = text_to_numbers(plaintext)

    # Mengubah urutan angka plaintext menjadi matriks dengan jumlah kolom sesuai dengan panjang key matrix
    plaintext_matrix = np.array(plaintext_numbers).reshape(-1, len(key_matrix))

    # Melakukan perkalian matriks plaintext dengan key matrix, kemudian diambil modulus 26
    encrypted_matrix = np.dot(plaintext_matrix, key_matrix) % 26

    # Mengonversi matriks hasil enkripsi menjadi array satu dimensi (bentuk angka berurut) [8, 7, 6, 25, 23, 21, 15, 9, 1, 4, 8, 12]
    encrypted_numbers = list(encrypted_matrix.flatten())

    # Mengonversi urutan angka hasil enkripsi menjadi teks dengan aturan 1=A, 2=B, ..., 26=Z
    ciphertext = numbers_to_text(encrypted_numbers, spaces)
    return ciphertext


# Fungsi untuk dekripsi dengan Hill Cipher
def hill_decrypt(ciphertext, key_matrix):
    spaces = [i for i, char in enumerate(ciphertext) if char == " "]
    ciphertext = ciphertext.replace(" ", "")

    ciphertext_numbers = text_to_numbers(ciphertext)

    # Mengubah bentuk angka berurut menjadi matriks. [8, 7, 6, 25, 23, 21, 15, 9, 1, 4, 8, 12] menjadi matriks 3 dimensi
    ciphertext_matrix = np.array(ciphertext_numbers).reshape(-1, len(key_matrix))

    # Mengapa perlu melakukan ini? Dalam cipher Hill, kita perlu mengecek apakah
    # GCD (Greatest Common Divisor) dari determinan dan 26 adalah 1, yang berarti
    # matriks kunci memiliki invers modulo 26. Tanpa invers modulo 26 dari key matrix,
    # pesan terenkripsi tidak dapat didekripsi kembali ke pesan asli.

    # Dengan kata lain, (a * x) % b = 1.
    # Misalnya, jika kita ingin mencari invers modulo 26 dari 5, kita mencari
    # bilangan bulat x sehingga (5 * x) % 26 = 1. Dalam hal ini, x adalah 21, karena (5 * 21) % 26 = 105 % 26 = 1.
    det = int(np.round(np.linalg.det(key_matrix)))

    if gcd(det, 26) == 1:
        # Menghitung invers modulo 26 dari key matrix
        # [[5, 6], [2, 3]] = 9 karena 3*9 % 26 = 27 % 26 = 1
        # Adjugate dari matriks 2x2 [[a, b], [c, d]] adalah [[d, -b], [-c, a]].
        # Maka [[3, -6], [-2, 5]]
        # Kali adjugate dengan invers modulo 26 dari determinan
        # Jadi, kita mendapatkan matriks [[27 % 26, -54 % 26], [-18 % 26, 45 % 26]],
        # yang setelah dihitung menjadi [[1, 0], [8, 19]].
        key_matrix_inv = Matrix(key_matrix).inv_mod(26)

        # Sisanya ke bawah sama penjelasannya kek fungnsi sebelumnya
        decrypted_matrix = np.dot(ciphertext_matrix, key_matrix_inv) % 26
        decrypted_numbers = list(decrypted_matrix.flatten())
        plaintext = numbers_to_text(decrypted_numbers, spaces)
        return plaintext
    else:
        raise ValueError("Matriks kunci tidak memiliki invers modulo 26.")


# Contoh penggunaan
key = np.array([[1, 2], [3, 1]])
plaintext = "AKU TOMAT"
ciphertext = hill_encrypt(plaintext, key)
print("Ciphertext:", ciphertext)

try:
    decrypted_text = hill_decrypt(ciphertext, key)
    print("Decrypted Text:", decrypted_text)
except ValueError as e:
    print(e)

Ciphertext: HMC JBQIV
Decrypted Text: AKU TOMAT


In [8]:
# Python3 code to implement Hill Cipher

keyMatrix = [[0] * 3 for i in range(3)]

# Generate vector for the message
messageVector = [[0] for i in range(3)]

# Generate vector for the cipher
cipherMatrix = [[0] for i in range(3)]


# Following function generates the
# key matrix for the key string
def getKeyMatrix(key):
    k = 0
    for i in range(3):
        for j in range(3):
            keyMatrix[i][j] = ord(key[k]) % 65
            k += 1


# Following function encrypts the message
def encrypt(messageVector):
    for i in range(3):
        for j in range(1):
            cipherMatrix[i][j] = 0
            for x in range(3):
                cipherMatrix[i][j] += keyMatrix[i][x] * messageVector[x][j]
            cipherMatrix[i][j] = cipherMatrix[i][j] % 26


def HillCipher(message, key):

    # Get key matrix from the key string
    getKeyMatrix(key)

    # Generate vector for the message
    for i in range(3):
        messageVector[i][0] = ord(message[i]) % 65

    # Following function generates
    # the encrypted vector
    encrypt(messageVector)

    # Generate the encrypted text
    # from the encrypted vector
    CipherText = []
    for i in range(3):
        CipherText.append(chr(cipherMatrix[i][0] + 65))

    # Finally print the ciphertext
    print("Ciphertext: ", "".join(CipherText))


# Driver Code
def main():

    # Get the message to
    # be encrypted
    message = "SSERANG"

    # Get the key
    key = "GYBNQKURP"

    HillCipher(message, key)


if __name__ == "__main__":
    main()

# This code is contributed
# by Pratik Somwanshi

Ciphertext:  YQY


In [9]:
import numpy as np
from sympy import Matrix


def gcd(a, b):
    while b:
        a, b = b, a % b
    return a


def text_to_numbers(text):
    return [(ord(char) - 65) % 26 + 1 for char in text.upper() if char != " "]


def numbers_to_text(numbers, spaces):
    text = "".join(chr((num - 1) % 26 + 65) for num in numbers)
    for index in spaces:
        text = text[:index] + " " + text[index:]
    return text


def hill_encrypt(plaintext, key_matrix):
    spaces = [i for i, char in enumerate(plaintext) if char == " "]
    plaintext = plaintext.replace(" ", "")
    plaintext_numbers = text_to_numbers(plaintext)
    plaintext_matrix = np.array(plaintext_numbers).reshape(-1, len(key_matrix))
    encrypted_matrix = np.dot(plaintext_matrix, key_matrix) % 26
    encrypted_numbers = list(encrypted_matrix.flatten())
    ciphertext = numbers_to_text(encrypted_numbers, spaces)
    return ciphertext


def hill_decrypt(ciphertext, key_matrix):
    spaces = [i for i, char in enumerate(ciphertext) if char == " "]
    ciphertext = ciphertext.replace(" ", "")
    ciphertext_numbers = text_to_numbers(ciphertext)
    ciphertext_matrix = np.array(ciphertext_numbers).reshape(-1, len(key_matrix))
    det = int(np.round(np.linalg.det(key_matrix)))

    if gcd(det, 26) == 1:
        key_matrix_inv = Matrix(key_matrix).inv_mod(26)
        decrypted_matrix = np.dot(ciphertext_matrix, key_matrix_inv) % 26
        decrypted_numbers = list(decrypted_matrix.flatten())
        plaintext = numbers_to_text(decrypted_numbers, spaces)
        return plaintext
    else:
        raise ValueError("Matriks kunci tidak memiliki invers modulo 26.")


key = np.array([[3, 5], [7, 11]])
plaintext = "ZAZABC"
ciphertext = hill_encrypt(plaintext, key)
print("Ciphertext:", ciphertext)

try:
    decrypted_text = hill_decrypt(ciphertext, key)
    print("Decrypted Text:", decrypted_text)
except ValueError as e:
    print(e)

Ciphertext: GKGKAQ
Matriks kunci tidak memiliki invers modulo 26.


In [10]:
import numpy as np
from sympy import Matrix


# Fungsi untuk menghitung GCD (Greatest Common Divisor)
def gcd(a, b):
    while b:
        a, b = b, a % b
    return a


# Fungsi untuk mengubah teks menjadi angka berdasarkan A=1, B=2, ..., Z=26
def text_to_numbers(text):
    return [ord(char) - 64 if char != " " else 0 for char in text.upper()]


# Fungsi untuk mengubah angka menjadi teks berdasarkan A=1, B=2, ..., Z=26
def numbers_to_text(numbers):
    return "".join(chr(num + 64) if num != 0 else " " for num in numbers)


# Fungsi untuk enkripsi dengan Hill Cipher
def hill_encrypt(plaintext, key_matrix):
    plaintext_numbers = text_to_numbers(plaintext)
    plaintext_matrix = np.array(plaintext_numbers).reshape(-1, len(key_matrix))
    encrypted_matrix = np.dot(plaintext_matrix, key_matrix) % 26
    encrypted_numbers = list(encrypted_matrix.flatten())
    ciphertext = numbers_to_text(encrypted_numbers)
    return ciphertext


# Fungsi untuk dekripsi dengan Hill Cipher
def hill_decrypt(ciphertext, key_matrix):
    ciphertext_numbers = text_to_numbers(ciphertext)
    ciphertext_matrix = np.array(ciphertext_numbers).reshape(-1, len(key_matrix))
    det = int(np.round(np.linalg.det(key_matrix)))
    if gcd(det, 26) == 1:
        key_matrix_inv = Matrix(key_matrix).inv_mod(26)
        decrypted_matrix = np.dot(ciphertext_matrix, key_matrix_inv) % 26
        decrypted_numbers = list(decrypted_matrix.flatten())
        plaintext = numbers_to_text(decrypted_numbers)
        return plaintext
    else:
        raise ValueError("Matriks kunci tidak memiliki invers modulo 26.")


# Contoh penggunaan
key = np.array([[5, 6], [2, 3]])
plaintext = "ZAZABC"
ciphertext = hill_encrypt(plaintext, key)
print("Ciphertext:", ciphertext)

try:
    decrypted_text = hill_decrypt(ciphertext, key)
    print("Decrypted Text:", decrypted_text)
except ValueError as e:
    print(e)

Ciphertext: BCBCPU
Decrypted Text:  A ABC
